# Retrieval augmented chat - Llama-2-7B-Chat

This notebook is the primary demonstration of the project with the baseline model. Here we'll bring up the baseline model and vector database and start asking questions both with and without the vector database.

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

## Initialize some variables

In [2]:
collection_name = "Corpus"
model_dir = "meta-llama/Llama-2-7b-chat-hf"
device_map = {"": 0}
device = "cuda"
database_top_n_results = 2

## Load shared code
This file defines the `ChatModel` and `Retrieval` classes used below.

In [3]:
%run shared_code.ipynb

## Load the model

In [4]:
language_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load the model into the ChatModel class from `shared_code.ipynb`

In [5]:
chat_model = ChatModel(language_model, tokenizer)

## Trying the model without access to the vector database.

In [6]:
chat_model.basic_chat("How do I loop a GIF?")

 To loop a GIF, you can use the `loop` attribute in the `<img>` tag. Here's an example:
```
<img src="your_gif_url" alt="Your Image Text" loop>
```
This will make the GIF repeat indefinitely. You can also set the duration of the loop by using the `duration` attribute, like this:
```
<img src="your_gif_url" alt="Your Image Text" loop duration="5">
```
In this case, the GIF will repeat for 5 seconds before repeating again.

Alternatively, if you want to loop the GIF only once, you can use the `once` attribute instead of `loop`:
```
<img src="your_gif_url" alt="Your Image Text" once>
```
Note that not all browsers support the `loop` and `once` attributes, so it's always a good idea to provide fallback options for non-supporting browsers.


## Load the collection into the RetrievalAugmentedChat class from `shared_code.ipynb`

In [7]:
rac = RetrievalAugmentedChat("db/", collection_name, database_top_n_results, chat_model)

## Run retrieval augmented chat
Notice that the responses have switched from a general HTML/JavaScript context to include the document contents which provide examples of encoding the gif with ImageMagick so that it loops by default.

In [8]:
rac.markdown_chat("How do I loop a GIF?")

 To loop a GIF using ImageMagick, you can use the `mogrify` tool with the `-loop` option. Here's an example of how to loop a GIF called `chrome-samesite-missing.gif`:

First, make sure you have ImageMagick installed on your system. On macOS, you can install ImageMagick using Homebrew:

```
brew install imagemagick
```
Next, open a terminal window and navigate to the directory where your GIF is located:

```
cd /path/to/your/directory
```
Now, run the following command to loop your GIF:

```
mogrify -loop 0 chrome-samesite-missing.gif
```
The `-loop 0` option tells ImageMagick to loop the GIF indefinitely. If you want to set the loop count explicitly, you can replace `0` with the desired value, such as `10`. For example:

```
mogrify -loop 10 chrome-samesite-missing.gif
```
This will loop the GIF 10 times.

Alternatively, you can use the `animate` command to loop the GIF:

```
animate chrome-samesite-missing.gif
```
This will also loop the GIF indefinitely.

You can also combine the `mogrify` and `animate` commands to loop the GIF while compressing it simultaneously:

```
mogrify -loop 0 -quality 70 chrome-samesite-missing.gif | animate
```
In this case, the `-quality 70` option sets the compression quality to 70%, which can help reduce the file size of the compressed GIF while maintaining its quality.

If you want to loop the GIF only part of the time, you can use the `montage` command with the `-loops` option:

```
montage -loops 50 chrome-samesite-missing.gif
```
This will loop the GIF every 50 milliseconds. Adjust the value of `-loops` to change the loop frequency.

Finally, if you want to create a new GIF file that loops continuously rather than modifying the existing one, you can use the `mogrify` command with the `-outfile` option:

```
mogrify -loop 0 -outfile looped_chrome-samesite-missing.gif chrome-samesite-missing.gif
```
This will create a new GIF file named `looped_chrome-samesite-missing.gif` that loops continuously.

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) distance: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) distance: 1.28

**Inference time in seconds 12.1914**


In [9]:
rac.markdown_chat("Can I use npx with GitHub actions?")

 Yes, you can use `npx` with GitHub Actions. In fact, you can leverage the built-in caching mechanism provided by GitHub Actions through the `actions/cache` action to improve the performance of your workflows. Here's how you can do it:

1. First, make sure you have the `actions/cache` action installed in your repository. You can install it by running the following command in your terminal:
```bash
npm install actions-cache
```
2. Next, define the `uses` section in your workflow file to use the `actions/cache` action. For example:
```yaml
uses: actions/cache @v2
```
This will enable the caching mechanism for all subsequent workflow steps.
3. To cache the output of a `npx` command, you can use the `cache` directive within the workflow step that invokes the command. For instance:
```yaml
- name: Install dependencies
  run:
    npx install --production &&
    cache:
      keys: ["dependencies"]
```
In this example, the `npx install --production` command is executed first, and its output is cached under the key "dependencies". If you run this workflow again, the next execution will skip the installation step and directly use the cached output instead.
4. To retrieve the cached output later in the same workflow, you can use the `useCache` directives followed by the path to the cached item. For example:
```yaml
- name: Run tests
  run:
    useCache:
      paths: ["dependencies"]
    npx test
```
In this scenario, the `test` command will be executed using the cached output of the `install` command.
5. Note that the `actions/cache` action does not support caching packages that are not present in the `package.json` file. Therefore, if you want to cache tools or scripts that are invoked using `npx`, you may need to specify the cache key manually using the `cache-key` property. For instance:
```yaml
cache-key: "custom-tool"
run:
  npx custom-tool > /dev/null
```
In this case, the output of the `npx custom-tool` command will be cached under the key "custom-tool".
6. Finally, keep in mind that the `actions/cache` action only caches outputs that are marked as `outputs`. Therefore, if you want to cache the output of a `npx` command that does not produce any explicit outputs, you may need to add an `outputs` section to your workflow step definition. For example:
```yaml
run:
  npx my-command &> /dev/null
  outputs:
    my-output: ${{ steps.my-command.outputs.my-output }}
```
In this scenario, the output of the `my-command` command will be captured and stored in the `my-output` variable, which can then be used as input to other workflow steps.

Overall, using `npx` with GitHub Actions can significantly improve the performance of your workflows by leveraging the built-in caching mechanism. Just remember to properly configure the `cache` and `useCache` directives to ensure that your workflows work correctly across different environments and scenarios.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) distance: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) distance: 1.11

**Inference time in seconds 14.5672**
